In [1]:
from soyspacing.countbase import CountSpace

model_fname = "./space-correct.model"
model = CountSpace()
model.load_model(model_fname, json_format=False)

In [2]:
import os
import re
import pandas as pd 
from eunjeon import Mecab

xname = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#7_dropped'}.xlsx")
df = pd.read_excel(xname, index_col=0)

tlist = []
blist = []

for i in range(len(df)):
    
    title = df.loc[i, 'title']
    body = df.loc[i, 'body']
    
    try:
        title = re.sub(r'[\(\[].*?[\)\]]', '', title)
    except:
        title = ''
        
    try:
        body = re.sub(r'[\(\[].*?[\)\]]', '', body)
    except:
        body = ''
        
    title = re.sub('【.*】','',title)
    title = re.sub(r'{.+}', '', title)
    title = re.sub(r'\d+?\.\s\d+\.\s\d+\.', '', title)
    title = re.sub(r'\d+\w+\d+', '', title)
    title = re.sub('[^0-9가-힣\.\n ]+', '', title)
    title = re.sub('\b[가-하]\.', '', title)
    title = re.sub(r'\b\w+법원\b', '', title)
    title = title.split('.')
    title = [e for e in title if len(e.strip()) > 2]
    title = [model.correct(e)[0] for e in title]
    trl = []
    for e in title:
        tr = ''
        for r in e.split():
            if len(r) > 1:
                if r.isalpha():
                    if tr == '':
                        tr = r
                    else:
                        tr = tr + " "+ r
        trl.append(re.sub(r'[0-9]', '', tr))
    trl = [e for e in trl if e]
    tlist = tlist + trl
    
    
    body = re.sub('【.*】','',body)
    body = re.sub(r'{.+}', '', body)
    body = re.sub(r'\d+?\.\s\d+\.\s\d+\.', '', body)
    body = re.sub(r'\d+\w+\d+', '', body)
    body = re.sub('[^0-9가-힣\.\n ]+', '', body)
    body = re.sub('\b[가-하]\.', '', body)
    body = re.sub(r'\b\w+법원\b', '', body)
    body = body.split('.')
    body = [e for e in body if len(e.strip()) > 2]
    body = [model.correct(e)[0] for e in body]
    brl = []
    for e in body:
        br = ''
        for r in e.split():
            if len(r) > 1:
                if r.isalpha():
                    if br == '':
                        br = r
                    else:
                        br = br + " "+ r
        brl.append(re.sub(r'[0-9]', '', br))
    brl = [e for e in brl if e]
    blist = blist + brl
    
    

    
mecab = Mecab()
text1 = " ".join(tlist)
text_list1 = mecab.nouns(text1)
word_list1 = pd.Series(text_list1)
t_64 = word_list1.value_counts().head(64)
t_64 = re.findall(r'\w+', t_64.to_string())
t_64 = [e for e in t_64 if e.isalpha()]


text2 = " ".join(blist)
text_list2 = mecab.nouns(text2)
word_list2 = pd.Series(text_list2)
b_128 = word_list2.value_counts().head(128)
b_128 = re.findall(r'\w+', b_128.to_string())
b_128 = [e for e in b_128 if e.isalpha()]

In [3]:
TEXT = []

for i in range(len(df)):
    title = df.loc[i, 'title']
    body = df.loc[i, 'body']
    
    try:
        title = re.sub(r'[\(\[].*?[\)\]]', '', title)
    except:
        title = ''
        
    try:
        body = re.sub(r'[\(\[].*?[\)\]]', '', body)
    except:
        body = ''
        
    
    title = re.sub('【.*】','',title)
    title = re.sub(r'{.+}', '', title)
    title = re.sub(r'\d+?\.\s\d+\.\s\d+\.', '', title)
    title = re.sub(r'\d+\w+\d+', '', title)
    title = re.sub('[^0-9가-힣\.\n ]+', '', title)
    title = re.sub('\b[가-하]\.', '', title)
    title = re.sub(r'\b\w+법원\b', '', title)
    title = title.split('.')
    title = [e for e in title if len(e.strip()) > 2]
    title = [model.correct(e)[0] for e in title]
    trl = []
    for e in title:
        tr = ''
        for r in e.split():
            if len(r) > 1:
                if r.isalpha():
                    if tr == '':
                        if mecab.nouns(r):
                            if mecab.nouns(r)[0] not in t_64:
                                tr = r
                        else:
                            tr = r
                    else:
                        if mecab.nouns(r):
                            if mecab.nouns(r)[0] not in t_64:
                                tr = tr + " "+ r
                        else:
                            tr = tr + " "+ r     
        trl.append(re.sub(r'[0-9]', '', tr))
    trl = [e for e in trl if e]
    
    
    body = re.sub('【.*】','',body)
    body = re.sub(r'{.+}', '', body)
    body = re.sub(r'\d+?\.\s\d+\.\s\d+\.', '', body)
    body = re.sub(r'\d+\w+\d+', '', body)
    body = re.sub('[^0-9가-힣\.\n ]+', '', body)
    body = re.sub('\b[가-하]\.', '', body)
    body = re.sub(r'\b\w+법원\b', '', body)
    body = body.split('.')
    body = [e for e in body if len(e.strip()) > 2]
    body = [model.correct(e)[0] for e in body]
    brl = []
    for e in body:
        br = ''
        for r in e.split():
            if len(r) > 1:
                if r.isalpha():
                    if br == '':
                        if mecab.nouns(r):
                            if mecab.nouns(r)[0] not in b_128:
                                br = r
                        else:
                            br = r
                    else:
                        if mecab.nouns(r):
                            if mecab.nouns(r)[0] not in b_128:
                                br = br + " "+ r
                        else:
                            br = br + " "+ r     
        brl.append(re.sub(r'[0-9]', '', br))
    brl = [e for e in brl if e]
    
    TEXT.append(trl+brl)

df['text'] = TEXT

df = df.drop(columns=['title', 'body'])
df = df[['url', 'cid', 'cbub', 'p1_age', 'p1_sex', 'p2_age', 'p2_sex', 'h1_age', 'h1_sex', 
         'h2_age', 'h2_sex', 'h3_age', 'h3_sex', 'case', 'law', 'text', 'appeal', 'y_sentence', 'y_probation', 'y_fine', 'y_appeal']]

x2name = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', f"{'xlsx_#8_clean_text'}.xlsx")
df.to_excel(x2name)

In [5]:
df[['h1_age', 'h1_sex']]

,h1_age,h1_sex
0,4,1
1,3,1
2,5,1
3,0,0
4,5,1
...,...,...
5374,3,2
5375,3,2
5376,3,2
5377,3,2
